In [18]:
import numpy as np
import scipy
from scipy.linalg import det

# Contents

* LU Decomposition
* QR Decomposition

## LU Decomposition

* A = LU
* det(A) = det(L) * det(U)

In [6]:
A = np.array([[7,3,-1,2],
              [3,8,1,-4],
              [-1,1,4,-1],
              [2,-4,-1,6]])

In [10]:
P, L, U = scipy.linalg.lu(A)

In [11]:
P  # Permutation

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [12]:
L # Lower-Left Triangle

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.42857143,  1.        ,  0.        ,  0.        ],
       [-0.14285714,  0.21276596,  1.        ,  0.        ],
       [ 0.28571429, -0.72340426,  0.08982036,  1.        ]])

In [14]:
U # Upper-Right Triangle

array([[ 7.        ,  3.        , -1.        ,  2.        ],
       [ 0.        ,  6.71428571,  1.42857143, -4.85714286],
       [ 0.        ,  0.        ,  3.55319149,  0.31914894],
       [ 0.        ,  0.        ,  0.        ,  1.88622754]])

In [20]:
assert det(L) * det(U) == det(A)

## LU로 연립방정식 풀기

* lu_factor, lu_solve

In [29]:
A = np.array([[7,3,-1,2],
              [3,8,1,-4],
              [-1,1,4,-1],
              [2,-4,-1,6]])
b = np.array([1,2,3,4])

lu, piv = scipy.linalg.lu_factor(A)

In [26]:
lu

array([[ 7.        ,  3.        , -1.        ,  2.        ],
       [ 0.42857143,  6.71428571,  1.42857143, -4.85714286],
       [-0.14285714,  0.21276596,  3.55319149,  0.31914894],
       [ 0.28571429, -0.72340426,  0.08982036,  1.88622754]])

In [27]:
piv

array([0, 1, 2, 3], dtype=int32)

In [30]:
# Ax = b, A = LU

x = scipy.linalg.lu_solve((lu, piv), b)
x

array([-1.27619048,  1.87619048,  0.57142857,  2.43809524])

In [34]:
# 수동으로 구하기 
#  x = A_inv * b = U_inv * L_inv * b

from scipy.linalg import inv

inv(U) @ inv(L) @ b

array([-1.27619048,  1.87619048,  0.57142857,  2.43809524])

In [35]:
# solver로 풀어보기

np.linalg.solve(A, b)

array([-1.27619048,  1.87619048,  0.57142857,  2.43809524])

## QR 분해

In [38]:
Q, R = np.linalg.qr(A)

In [39]:
Q # 직교행렬

array([[-0.8819171 ,  0.12001372, -0.10823812, -0.44283568],
       [-0.37796447, -0.75437193,  0.06201752,  0.53312257],
       [ 0.12598816, -0.18859298, -0.97180277, -0.06449063],
       [-0.25197632,  0.6172134 , -0.20009425,  0.71799572]])

In [40]:
R

array([[-7.93725393, -4.53557368,  1.25988158, -1.88982237],
       [ 0.        , -8.3323809 , -2.24597098,  7.14938855],
       [ 0.        ,  0.        , -3.51686121, -0.69330902],
       [ 0.        ,  0.        ,  0.        ,  1.35430329]])

In [42]:
# 직교행렬 특징 검증하기 

I = np.eye(A.shape[0])

np.allclose( Q @ Q.T, I )   # Q * Q_T = Identity

True

In [44]:
np.allclose( inv(Q.T), Q )  # Q_inv = Q_T

True

## Eigen decomposition

In [98]:
# Ax = lambda*x

A = np.array([[1,3],[3,1]])

In [99]:
e_val, e_vec = np.linalg.eig(A)
e_val, e_vec 

(array([ 4., -2.]),
 array([[ 0.70710678, -0.70710678],
        [ 0.70710678,  0.70710678]]))

In [105]:
# 검증 

lambda1 = e_val[0]
lambda2 = e_val[1]

E = np.eye(A.shape[0])

assert np.allclose( det( A - (lambda1*E) ), 0)
assert np.allclose( det( A - (lambda2*E) ), 0)

In [109]:
x2 = np.squeeze(e_vec[0,:])   # 왜 순서가 반대이지?
x1 = np.squeeze(e_vec[1,:])

assert np.allclose( A @ x1, lambda1 * x1 )
assert np.allclose( A @ x2, lambda2 * x2 )

## SVD 분해

In [110]:
A = np.array([[1,0],[1,2]])

In [111]:
U, S, V = scipy.linalg.svd(A)

In [112]:
U

array([[-0.22975292, -0.97324899],
       [-0.97324899,  0.22975292]])

In [113]:
S

array([2.28824561, 0.87403205])

In [114]:
V

array([[-0.52573111, -0.85065081],
       [-0.85065081,  0.52573111]])

In [118]:
# 검증 
## A = USV

E = np.eye(A.shape[0])
SE = S * E 

np.allclose( U @ SE @ V , A )

True

# 희소행렬

## CSR 형식

In [128]:
row = np.array([0,0,1,2,2,2])
col = np.array([0,2,2,0,1,2])
data = np.array([1,2,3,4,5,6])

csr = scipy.sparse.csr_matrix((data, (row,col)), shape=(3,3))
csr

<3x3 sparse matrix of type '<class 'numpy.longlong'>'
	with 6 stored elements in Compressed Sparse Row format>

In [129]:
csr.shape, csr.ndim

((3, 3), 2)

In [130]:
csr.indices, csr.indptr

(array([0, 2, 2, 0, 1, 2], dtype=int32), array([0, 2, 3, 6], dtype=int32))

In [131]:
csr.toarray()

array([[1, 0, 2],
       [0, 0, 3],
       [4, 5, 6]], dtype=int64)

## CSC 형식

In [133]:
x = np.array([[1,0,0,0],
              [0,3,0,0],
              [0,1,1,0],
              [1,0,0,1]])

In [135]:
csc = scipy.sparse.csc_matrix(x)
csc

<4x4 sparse matrix of type '<class 'numpy.longlong'>'
	with 6 stored elements in Compressed Sparse Column format>

In [136]:
csc.toarray()

array([[1, 0, 0, 0],
       [0, 3, 0, 0],
       [0, 1, 1, 0],
       [1, 0, 0, 1]], dtype=int64)

## LIL 양식

In [140]:
x = np.array([[1,0,0,0],
              [0,3,0,0],
              [0,1,1,0],
              [1,0,0,1]])

In [141]:
lil = scipy.sparse.lil_matrix(x)
lil

<4x4 sparse matrix of type '<class 'numpy.longlong'>'
	with 6 stored elements in List of Lists format>

In [142]:
lil.toarray()

array([[1, 0, 0, 0],
       [0, 3, 0, 0],
       [0, 1, 1, 0],
       [1, 0, 0, 1]], dtype=int64)